# Bertoluzzo (2007) — Intradía con umbrales y ajuste tipo RL
Estrategia intradía basada en umbrales con ajuste online del umbral vía gradiente sobre PnL suavizado. Baseline pragmático que emula la filosofía del paper: simplicidad y robustez.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
rng = np.random.default_rng(7)


In [ ]:

def intraday_returns(n=3000, sigma=0.0015, bursts=0.003, p_burst=0.02, seed=7):
    rng = np.random.default_rng(seed)
    r = rng.normal(0, sigma, size=n)
    mask = rng.random(n) < p_burst
    r[mask] += rng.normal(0, bursts, size=mask.sum())
    return r

r = intraday_returns()


In [ ]:

def signals_threshold(r, theta):
    sig = np.zeros_like(r)
    sig[r > theta] = 1
    sig[r < -theta] = -1
    return sig

def pnl_with_cost(sig, r, cost=0.00005):
    ds = np.diff(sig, prepend=0)
    pnl = sig*r - cost*np.abs(ds)
    return pnl

theta = 0.001
lr = 0.1
for epoch in range(10):
    sig = signals_threshold(r, theta)
    pnl = pnl_with_cost(sig, r)
    # proxy grad: d pnl / d theta ≈ -sum( δ(r - theta) * r )  -> aproximar con banda suave
    band = 200.0
    edge_up = 0.5*(1+np.tanh(band*(r - theta)))
    edge_dn = 0.5*(1+np.tanh(band*(-r - theta)))
    # sensibilidad de señal a theta (aprox)
    d_sig_d_theta = - (1 - edge_up**2) * band + - (1 - edge_dn**2) * band
    grad = np.mean(d_sig_d_theta * r)  # muy heurístico pero estable para tuning
    theta += lr * grad
    theta = np.clip(theta, 1e-5, 0.01)

sig = signals_threshold(r, theta)
pnl = pnl_with_cost(sig, r)
print("theta final:", theta, " SR:", pnl.mean()/(pnl.std()+1e-9))

plt.figure(); plt.plot(np.cumsum(pnl)); plt.title('Equity curve (threshold strategy)'); plt.show()
